In [1]:
!pip install -q langchain-openai langchain

from google.colab import userdata
from langchain_openai import OpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableLambda
from langchain.schema import HumanMessage, AIMessage

# 1. API key retrieval and LLM initialisation
OPENAI_API_KEY = userdata.get("OpenAI-key")
llm = OpenAI(api_key=OPENAI_API_KEY)

# 2. Memory initialisation
memory = ConversationBufferWindowMemory(k=2, return_messages=True)

# 3. Prompt template
template = """
You are a helpful assistant. Use the provided conversation history to answer the user's questions.
{history}
Human: {input}
AI:"""
prompt = PromptTemplate(input_variables=["history", "input"], template=template)

# 4. Memory loading and saving functions
def memory_load(input_data):
    return memory.load_memory_variables(input_data)["history"]

def add_message(input_output):
    memory.save_context({"input": input_output["input"]}, {"output": input_output["output"]})
    return input_output["output"]

# 5. Chain construction (RunnableSequence)
chain = (
    {"input": RunnablePassthrough(), "history": RunnableLambda(memory_load)}
    | prompt
    | llm
    | RunnableLambda(lambda x: {"input": input_data["input"], "output":x})
    | RunnableLambda(add_message)
)

# 6. Example conversation
input_data = {"input": "Hello, I'm Alice."}
result1 = chain.invoke(input_data)
print(result1)

input_data = {"input": "What is my name?"}
result2 = chain.invoke(input_data)
print(result2)

input_data = {"input": "What was the first thing I said?"}
result3 = chain.invoke(input_data)
print(result3)

input_data = {"input": "What was the second thing I said?"}
result4 = chain.invoke(input_data)
print(result4)

# 7. Memory inspection
print(memory.load_memory_variables({}))

<ipython-input-1-d570ed24850e>:16: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=2, return_messages=True)


 Hello Alice, how can I assist you?
 Your name is Alice.
 You said "Hello, I'm Alice."
 You said "Your name is Alice."
{'history': [HumanMessage(content='What was the first thing I said?', additional_kwargs={}, response_metadata={}), AIMessage(content=' You said "Hello, I\'m Alice."', additional_kwargs={}, response_metadata={}), HumanMessage(content='What was the second thing I said?', additional_kwargs={}, response_metadata={}), AIMessage(content=' You said "Your name is Alice."', additional_kwargs={}, response_metadata={})]}
